# Image to .AFF Trace Converter
**By: rinH\***

Version: Test1

---



In [ ]:
#@title /!\ ***RUN THIS CELL FIRST*** /!\
#@markdown ---
#@markdown A link will be generated when run, click the link generated and sign in with your Google Drive account to obtain your verification id. Then copy the id to the textbox in the console.

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import re

from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive'

In [ ]:
#@title Import Image

#@markdown Insert the name of the image in the textbox below (MUST BE A .JPG STORED IN THE ROOT DIRECTORY OF YOUR GOOGLE DRIVE):
img_file_name = 'im.jpg' #@param {type:"string"}
img = cv.imread(img_file_name) # Image needs to be a JPG file and need to be stored in
                             # the root directory of your Google drive

#@markdown ---
#@markdown #### Resize image
#@markdown The lower the image size, the lower the number of AFF lines will be generated. Lowering value may also reduce the amount of detail.
img_size_ratio = 0.2 #@param {type:"slider", min:0.01, max:1, step:0.01}

#Adjust image height and width, lower values lowers the number of lines in arcade
target_h = int(img.shape[0] * img_size_ratio)
target_w = int(img.shape[1] * img_size_ratio)

#@markdown ---
#@markdown #### Adjust Trace size
arc_h = 1 #@param {type:"number"}
arc_w = 2 #@param {type:"number"}

#@markdown ---
#@markdown #### Adjust arc origin (0, 0) point [Bottom left corner of the trace art]
arc_x_origin = -0.5 #@param{type:"number"}
arc_y_origin =  0#@param{type:"number"}

#@markdown ---
print("Original Size" + str(img.shape)) # Prints the original size of the image

### Image Preprocessing ###
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) # Change color of image to grayscale
img_resized = cv.resize(img_gray, (target_w, target_h), interpolation = cv.INTER_AREA) # Resize image to values specified

edges_canny = cv.Canny(img_resized,100,150) # Edge detection

plt.imshow(edges_canny) # Shows the edges
print("Compressed Size" + str(edges_canny.shape))

### Generate Chart File

In [ ]:
#@markdown Execute this cell to generate traces and download the trace's AFF file

flipped = cv.flip(edges_canny, 0) # idk why it gives a flipped image in Arcade for some reason

connected = flipped / 255 # Normalize values for connecting the pixels
print(connected.shape)

pairs = [] # store pairs of coordinates []

for y in range(target_h): # loops through each pixel of the image.
  for x in range(target_w):
    if connected[y][x] == 1: # if a pixel is 1, then an edge is detected
      first_coor = x, y
      connected[y][x] = 0 # set the pixel to connected
      for i in range(-1, 2):
        for j in range(-1, 2):
          if (y + i) < target_h and (x + j) < target_w and (y + i) >= 0 and (x + j) >= 0:
            if connected[y + i][x + j] == 1:
              connected_coor = (x + j), (y + i)
              paired_coor = first_coor, connected_coor
              pairs.append(paired_coor)
print(pairs)

from google.colab import files
with open('arc.txt', 'w') as f:
    for pair in pairs:
      x_start = str((pair[0][0] * arc_w / target_w) + arc_x_origin)
      x_end = str((pair[1][0] * arc_w / target_w) + arc_x_origin)
      y_start = str((pair[0][1] * arc_h / target_h) + arc_y_origin)
      y_end = str((pair[1][1] * arc_h / target_h) + arc_y_origin)
      f.write("arc(1000,1000,"+ x_start +","+ x_end +",s,"+ y_start +","+ y_end +",0,none,true);\n")

files.download('arc.txt')
print('arc.txt file downloaded. Copy the file content into your chart file.')